# Hierarchical Clustering in GenePattern Notebook

Cluster genes and/or samples based on how close they are to one another. The result is a tree structure, referred to as dendrogram.

## Before you begin

* Sign in to GenePattern by entering your username and password into the form below.
* Gene expression data must be in a [GCT or RES file](https://genepattern.broadinstitute.org/gp/pages/protocols/GctResFiles.html) - we have provided files in the correct format.
    * Example file: [all_aml_test.gct](https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct).
* Learn more by reading about [file formats](http://www.broadinstitute.org/cancer/software/genepattern/file-formats-guide#GCT).


In [5]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.GPAuthWidget(genepattern.register_session("https://genepattern.broadinstitute.org/gp", "", ""))

## Step 1: PreprocessReadCounts

Preprocess RNA-Seq count data in a GCT file so that it is suitable for use in GenePattern analyses. 

### Introduction

* This module is used to preprocess RNA-Seq data into a form suitable for use downstream in other GenePattern analyses such as Gene Set Enrichment Analysis (GSEA), ComparativeMarkerSelection, HierarchicalClustering, as well as visualizers. 
* Many of these approaches assume that the data is distributed normally, yet this is not true of RNA-seq read count data. The PreprocessReadCounts module provides one approach to accommodate this. It uses a mean-variance modeling technique to transform the dataset to fit an approximation of a normal distribution, with the goal of being able to apply statistical methods and workflows that assume a normal distribution.
* Learn more by reading about the [PreprocessReadCounts](http://software.broadinstitute.org/cancer/software/genepattern/modules/docs/PreprocessReadCounts/1) module.

<div class="alert alert-info">
<h3>Instructions</h3>

(NOTE: if clicking and dragging the links does not work, right-click the links and save them locally, and upload them by clicking **Upload File...** in the corresponding input box)

<ol>
<li>For the <strong><em>input.file</em></strong> parameter, click and drag <a href="counts_for_deseq2_genes.gct" target="_blank">counts_for_deseq2_genes.gct</a> into the <em>&quot;Enter Path or URL&quot; </em>text box</li>         <li>For the <strong><em>cls.file</em></strong> parameter, click and drag <a href="FTO_vs_NT.cls"target="_blank">FTO_vs_NT.cls</a> into the <em>&quot;Enter Path or URL&quot; </em>text box</li>   <li>Set the <strong><em>expression.value.filter.threshold</em></strong> parameter to 4 (so as to reduce the number of rows, and thus the time for computation)</li>
<li>Click the button <strong><em>Run</em></strong> on the analysis below.</li>
</ol>

</div>


In [8]:
preprocessreadcounts_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00355')
preprocessreadcounts_job_spec = preprocessreadcounts_task.make_job_spec()
preprocessreadcounts_job_spec.set_parameter("input.file", "https://genepattern.broadinstitute.org/gp/users/mmrtest5/tmp/run4740190127956208280.tmp/counts_for_deseq2_genes.gct")
preprocessreadcounts_job_spec.set_parameter("cls.file", "https://genepattern.broadinstitute.org/gp/users/mmrtest5/tmp/run2566002573629637303.tmp/FTO_vs_NT.cls")
preprocessreadcounts_job_spec.set_parameter("output.file", "<input.file_basename>.preprocessed.gct")
preprocessreadcounts_job_spec.set_parameter("expression.value.filter.threshold", "4")
genepattern.GPTaskWidget(preprocessreadcounts_task)

## Step 2: HierarchicalClustering

Run hierarchical clustering on genes and/or samples to create dendrograms for the clustered genes (*.gtr) and/or clustered samples (*.atr), as well as a file (*.cdt) that contains the original gene expression data ordered to reflect the clustering.

### Considerations
* Best practice is to normalize (row/column normalize parameters) and center (row/column center parameters) the data being clustered. 
* The CDT output file must be converted to a GCT file before it can be used as an input file for another GenePattern module (other than HierachicalClusteringViewer). For instructions on converting a CDT file to a GCT file, see [Creating Input Files](http://www.broadinstitute.org/cancer/software/genepattern/file-formats-guide#creating-input-files).
* Learn more by reading about the [HierarchicalClustering](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=HierarchicalClustering) module.

<div class="alert alert-info">
**Instructions**
<p>
After the PreprocessReadCounts job above finishes running, send the GCT result of that job to HierarchicalClustering below.<br>
To do this, you can do **either** of the following:
<ol>
<li> Drag and drop the link for the file above to the *input.filename* input below, **OR**<br>
 Click the link for the file above and select *Send to an Existing GenePattern Cell > HierarchicalClustering* in the menu that appears.</li>
<li> For the "column distance measure" parameter, select "No column clustering"</li>
<li> Once this is done, click *Run* for the analysis below.</li>
</ol>


</div>

In [6]:
hierarchicalclustering_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00009')
hierarchicalclustering_job_spec = hierarchicalclustering_task.make_job_spec()
hierarchicalclustering_job_spec.set_parameter("input.filename", "https://genepattern.broadinstitute.org/gp/jobResults/1555856/counts_for_deseq2_genes.preprocessed.gct")
hierarchicalclustering_job_spec.set_parameter("column.distance.measure", "0")
hierarchicalclustering_job_spec.set_parameter("row.distance.measure", "2")
hierarchicalclustering_job_spec.set_parameter("clustering.method", "a")
hierarchicalclustering_job_spec.set_parameter("log.transform", "")
hierarchicalclustering_job_spec.set_parameter("row.center", "mean.row")
hierarchicalclustering_job_spec.set_parameter("row.normalize", "")
hierarchicalclustering_job_spec.set_parameter("column.center", "mean.column")
hierarchicalclustering_job_spec.set_parameter("column.normalize", "")
hierarchicalclustering_job_spec.set_parameter("output.base.name", "<input.filename_basename>")
genepattern.GPTaskWidget(hierarchicalclustering_task)

## Step 3: HierarchicalClusteringViewer

Display a heat map of the clustered gene expression data, with dendrograms showing how the genes and/or samples were clustered.

### Considerations

* Select File > Save Image to save the heat map and dendrograms to an image file. Supported formats include bmp, eps, jpeg, png, and tiff. 
* Learn more by reading about the [HierarchicalClusteringViewer](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=HierarchicalClusteringViewer) module.

<div class="alert alert-info">
<h3>Instructions</h3>

<p>After the HierarchicalClustering job above finishes running, send the CDT, GTR and ATR results of that job to HierarchicalClustering below. You will need to scroll back to the HierarchicalClustering job after sending the first file. Once this is done, click <em>Run</em> for the analysis below. <em>(note that the HierarchicalClustering job will take ~10-15 minutes to complete)</em></p>
</div>

In [7]:
hierarchicalclusteringviewer_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00031')
hierarchicalclusteringviewer_job_spec = hierarchicalclusteringviewer_task.make_job_spec()
hierarchicalclusteringviewer_job_spec.set_parameter("cdt.file", "")
hierarchicalclusteringviewer_job_spec.set_parameter("gtr.file", "")
hierarchicalclusteringviewer_job_spec.set_parameter("atr.file", "")
genepattern.GPTaskWidget(hierarchicalclusteringviewer_task)